# **Assignment No :- 3**
## **Fine-tune GPT or GPT-2 for creative story generation.**

In [1]:
# Install Libraries
pip install torch transformers datasets

In [7]:
# Import Libraries
import torch
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)


In [8]:
# Load GPT-2 Tokenizer + Model
model_name = "gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token   # IMPORTANT

model = GPT2LMHeadModel.from_pretrained(model_name)

print("✅ Model and tokenizer loaded!")


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Model and tokenizer loaded!


In [9]:
# Load Dataset (story_dataset.txt)
dataset = load_dataset("text", data_files={"train": "story_dataset.txt"})
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 19
    })
})


In [10]:
# Tokenization (Convert text → tokens)
MAX_LEN = 256

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

print("✅ Tokenization completed!")
print(tokenized_dataset)


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

✅ Tokenization completed!
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 19
    })
})


In [14]:
# Data Collator (Batching for GPT-2)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

print(" Data collator created!")


 Data collator created!


In [15]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-stories",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none",
    fp16=torch.cuda.is_available()
)

print(" Training arguments set!")


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


 Training arguments set!


In [16]:
# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)

print("Trainer initialized!")

Trainer initialized!


In [17]:
# Start Training
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=15, training_loss=3.0177342732747396, metrics={'train_runtime': 206.4155, 'train_samples_per_second': 0.276, 'train_steps_per_second': 0.073, 'total_flos': 7446822912000.0, 'train_loss': 3.0177342732747396, 'epoch': 3.0})

In [19]:
# Save Fine-Tuned Model
model.save_pretrained("./gpt2-finetuned-stories")
tokenizer.save_pretrained("./gpt2-finetuned-stories")

print("Fine-tuning complete! Model saved.")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Fine-tuning complete! Model saved.


# **STORY GENERATION PART (After Training)**

In [21]:
# Load Fine-Tuned Model for Generation
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import textwrap

model_name = "./gpt2-finetuned-stories"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Fine-tuned model loaded for story generation!")


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

Fine-tuned model loaded for story generation!


In [22]:
# Story Generator Function
def generate_story(prompt, max_length=250):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

    story = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Formatting (no right-side scrolling)
    story = story.replace(". ", ".\n")
    return textwrap.fill(story, width=90)


In [23]:
# Generate Story Output
story_prompt = "Once upon a time in a mysterious forest,"
print(generate_story(story_prompt))


Once upon a time in a mysterious forest, there was only one man who could save the world.
He had found himself trapped between two worlds—one that seemed to be destined for greater
things and another where he would forever live among humanity's most ancient beasts. - - -
- Grimoire of Heroes is an adventure set across centuries long ago when heroes discover
their place amongst countless races throughout history. But once they arrived on this
land, it became impossible not meet them all: powerful but weak warriors with strong minds
imbued with magic like never before... And those mages were determined to protect her from
evil even against darkness! As you approach Grimoires Island beneath waves of wind, find
out if your journey will change anything about fate or help restore order through destiny!
